# Overview

We will be doing the following to create a Deep Neural Network using RNN and Softmax as the activation output layer:

- Instantiate required Python components.
- Set Hyperparameters
- Read the CSV data
- Remove unused fields.
- Keep only the message in the JSON.
- Define two lists: messages and labels.
- Split data between training and validation sets.
- Tokenize words
- Pad sequences so they are the same size.
- Build LSTM
- Train several epochs.
- Plot Loss and Accuracy to view model's performance.
- Make predictions.


# Instantiate required Python components.

Our project will use TensorFlow for developing our model.  We'll also need several other Python libraries to work with our CSV.

In [ ]:
import csv
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

# Set Hyperparameters

This handy section will control all the important parameters for our model.

In [1]:
vocab_size = 5000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = ''
training_portion = .8

# Read the CSV data

In [ ]:
labels = []
messages = []







# Remove unused fields.

# Keep only the message in the JSON.